In [1]:
from dataset_facemask import *
from model_facemask import *
from torchvision import transforms
from torchsummary import summary


In [2]:
learning_rate=2e-4
batch_size=8

In [3]:
train=['D:/data/train_data_ia.npy',
       'D:/data/train_data_ib.npy',
       'D:/data/train_big_pose_a.npy',
       'D:/data/train_big_pose_b.npy',
       'D:/data/train_data_pa_mask_full.npy',
       'D:/data/train_data_pb_mask_full.npy',
       'D:/data/train_data_gray_b.npy'
      ]
test=['D:/data/test_data_ia.npy',
      'D:/data/test_data_ib.npy',
      'D:/data/test_big_pose_a.npy',
      'D:/data/test_big_pose_b.npy',
      'D:/data/test_data_pa_mask_full.npy',
      'D:/data/test_data_pb_mask_full.npy',
      'D:/data/test_data_gray_b.npy']



In [4]:
composed = transforms.Compose([ToTensor()])

# train_dataset=ImageDataset(fnames=test,transform=composed)
test_dataset=ImageDataset(fnames=train,transform=composed)
# train_loader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=batch_size,shuffle=False)


In [3]:
# gen=Generator().cuda()
gen=UNet_generator().cuda()
dis1=Discriminator1().cuda()
dis2=Discriminator2().cuda()
criterion=nn.MSELoss()
gen_optimizer=torch.optim.Adam(gen.parameters(),lr=learning_rate)
dis1_optimizer=torch.optim.Adam(dis1.parameters(),lr=learning_rate)
dis2_optimizer=torch.optim.Adam(dis2.parameters(),lr=learning_rate)

In [ ]:
# # ,(8,5, 128, 128),(8,5, 128, 128)
# summary(gen,(13,128, 128))
# summary(dis1,(6,128, 128))
# summary(dis2,(8,128, 128))


In [9]:
#load pretrained data using lightCNN 29layers
pretrained_dict=torch.load('LightCNN_29Layers_checkpoint.pth.tar')
lightcnnmodel=network_29layers(resblock, [1, 2, 3, 4]).cuda()
lightcnnmodel_dict = lightcnnmodel.state_dict()
# # 1. filter out unnecessary keys
pretrained_dict = {k.replace("module.",""): v for k, v in pretrained_dict['state_dict'].items() if k.replace("module.","") in lightcnnmodel_dict}
# # 2. overwrite entries in the existing state dict
lightcnnmodel_dict.update(pretrained_dict) 
# # 3. load the new state dict
lightcnnmodel.load_state_dict(pretrained_dict)

In [ ]:
# summary(lightcnnmodel,(1,128, 128))


In [7]:
# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
def reset_grad():
    gen_optimizer.zero_grad()
    dis1_optimizer.zero_grad()
    dis2_optimizer.zero_grad()
def denorm(x):
    out = (x+1)/2
    return out.clamp(0, 1)


In [8]:
l1_loss = nn.L1Loss()

In [9]:
def training(learning_rate=2e-4, num_epochs=200):
    gen.load_state_dict(torch.load("saver/mask/mask_G.ckpt"))
    dis1.load_state_dict(torch.load("saver/mask/mask_D1.ckpt"))
    dis2.load_state_dict(torch.load("saver/mask/mask_D2.ckpt"))
    #training step
    curr_lr = learning_rate
    start = time.time()
    total_step = len(train_loader)
    
    for epochs in range(num_epochs):
        for i, (ia,ib,pa,pb,pam,pbm,gray_b)in enumerate(train_loader):

            
            image_a = Variable(ia).cuda()
            image_b = Variable(ib).cuda() 
            pose_a = Variable(pa).cuda()
            pose_b = Variable(pb).cuda()
            posemask_a = Variable(pam).cuda()
            posemask_b = Variable(pbm).cuda()
            gray_b = Variable(gray_b).cuda()
            i_num=len(image_a)
            #real label fake label
            real_labels = torch.ones(1, i_num).cuda()
            fake_labels = torch.zeros(1,i_num).cuda()

            '''
            real image 인지 
            dis_ii_val1 = D1(ib,ia)
            dis_ii_val2 = D1(i^b,ia)

            real pose 인지
            dis_pe_val1=D2(ib,pb)
            dis_pe_val2=D2(i^b,pb)
            '''


            #discriminator1 학습 --> 이미지의 형태를 판별
            gen_image,gen_gray_img= gen.forward(image_a,pose_a,pose_b)      
            #얘네를 참으로 학습하게한다
            dis_ii_val1=dis1.forward(image_b,image_a)
            dis_ii_val2=dis1.forward(gen_image,image_a)
            dis_pe_val1=dis2.forward(image_b,pose_b)
            dis_pe_val2=dis2.forward(gen_image,pose_b)
#             print(dis_ii_val1.shape,real_labels.shape,fake_labels.shape)


            #light cnn feature extractor

            D_pool_i_gb,D_fc_i_gb=lightcnnmodel.forward(gen_gray_img)
            D_pool_i_b,D_fc_i_b=lightcnnmodel.forward(gray_b)

            loss_pix_t= multi_scale_pixel_wise_loss(image_b,gen_image)
            loss_ii_adv_t=criterion(dis_ii_val1,real_labels)
            loss_pe_adv_t=0
            loss_ip_t=identity_preserving_loss(D_pool_i_gb,D_pool_i_b,D_fc_i_gb,D_fc_i_b)
            loss_tv_t=total_variant_regularization(gen_image)
            real_total_loss=total_loss(loss_pix_t,loss_ii_adv_t,loss_pe_adv_t,loss_ip_t,loss_tv_t)
            
            loss_pix_f= multi_scale_pixel_wise_loss(image_b,gen_image)
            loss_ii_adv_f=criterion(dis_ii_val2,fake_labels)
            loss_pe_adv_f=0
            loss_ip_f=identity_preserving_loss(D_pool_i_gb,D_pool_i_b,D_fc_i_gb,D_fc_i_b)
            loss_tv_f=total_variant_regularization(gen_image)
            fake_total_loss=total_loss(loss_pix_f,loss_ii_adv_f,loss_pe_adv_f,loss_ip_f,loss_tv_f)
            
            d1_total_loss=real_total_loss+fake_total_loss
            # Backprop and optimize
            dis1_optimizer.zero_grad()
            d1_total_loss.backward()
            dis1_optimizer.step()


            #discriminator2 학습 --> pose 판별
            gen_image,gen_gray_img= gen.forward(image_a,pose_a,pose_b)    
            #얘네를 참으로 학습하게한다

            dis_ii_val1=dis1.forward(image_b,image_a)
            dis_ii_val2=dis1.forward(gen_image,image_a)
            dis_pe_val1=dis2.forward(image_b,pose_b)
            dis_pe_val2=dis2.forward(gen_image,pose_b)

            
            #light cnn feature extractor

            D_pool_i_gb,D_fc_i_gb=lightcnnmodel.forward(gen_gray_img)
            D_pool_i_b,D_fc_i_b=lightcnnmodel.forward(gray_b)

            loss_pix_t = multi_scale_pixel_wise_loss(image_b,gen_image)
            loss_ii_adv_t=0
            loss_pe_adv_t=criterion(dis_pe_val1,real_labels)
            loss_ip_t=identity_preserving_loss(D_pool_i_gb,D_pool_i_b,D_fc_i_gb,D_fc_i_b)
            loss_tv_t=total_variant_regularization(gen_image)        
            real_total_loss=total_loss(loss_pix_t,loss_ii_adv_t,loss_pe_adv_t,loss_ip_t,loss_tv_t)

            loss_pix_f = multi_scale_pixel_wise_loss(image_b,gen_image)
            loss_ii_adv_f=0
            loss_pe_adv_f=criterion(dis_pe_val2,fake_labels)
            loss_ip_f=identity_preserving_loss(D_pool_i_gb,D_pool_i_b,D_fc_i_gb,D_fc_i_b)
            loss_tv_f=total_variant_regularization(gen_image)        
            fake_total_loss=total_loss(loss_pix_f,loss_ii_adv_f,loss_pe_adv_f,loss_ip_f,loss_tv_f)

            d2_total_loss=real_total_loss+fake_total_loss
            dis2_optimizer.zero_grad()
            d2_total_loss.backward()
            dis2_optimizer.step()

            #gen_image's shape : width :
            gen_image,gen_gray_img= gen.forward(image_a,pose_a,pose_b)     
            #얘네를 참으로 학습하게한다
            dis_ii_val1=dis1.forward(image_b,image_a)
            dis_ii_val2=dis1.forward(gen_image,image_a)
            dis_pe_val1=dis2.forward(image_b,pose_b)
            dis_pe_val2=dis2.forward(gen_image,pose_b)
            posemask_loss=pose_mask_loss(gen_image,image_b,posemask_b)

            #generator
            
            D_pool_i_gb,D_fc_i_gb=lightcnnmodel.forward(gen_gray_img)
            D_pool_i_b,D_fc_i_b=lightcnnmodel.forward(gray_b)

            loss_pix_t = multi_scale_pixel_wise_loss(image_b,gen_image)
            loss_ii_adv_t=criterion(dis_ii_val2,real_labels)
            loss_pe_adv_t=criterion(dis_pe_val2,real_labels)
            loss_ip_t=identity_preserving_loss(D_pool_i_gb,D_pool_i_b,D_fc_i_gb,D_fc_i_b)
            loss_tv_t=total_variant_regularization(gen_image)
            g_total_loss=total_loss(loss_pix_t,loss_ii_adv_t,loss_pe_adv_t,loss_ip_t,loss_tv_t)+posemask_loss

            # Backprop and optimize
            gen_optimizer.zero_grad()
            g_total_loss.backward()
            gen_optimizer.step()


    #         if (i+1) % 10 == 0:
        print('Epoch [{}/{}], Step [{}/{}], d1_loss: {:.4f},d2_loss: {:.4f}, g_total_loss: {:.4f}'
              .format(epochs, num_epochs, i+1, total_step, d1_total_loss.mean(), d2_total_loss.mean(),
                      g_total_loss.mean()))
        # Decay learning rate
        if (epochs+1) % 20 == 0:
            curr_lr /= 3
            update_lr(gen_optimizer, curr_lr)
            update_lr(dis1_optimizer, curr_lr)
            update_lr(dis2_optimizer, curr_lr)
        if (epochs+1) % 10 == 0:
            torch.save(gen.state_dict(),'saver/mask/mask_G.ckpt')
            torch.save(dis1.state_dict(),'saver/mask/mask_D1.ckpt')
            torch.save(dis2.state_dict(),'saver/mask/mask_D2.ckpt')

        save_image(denorm(gen_image),'samples/mask/gen_images-{}.png'.format(epochs+1))
        save_image(denorm(image_a),'samples/mask/a_image-{}.png'.format(epochs+1))
        save_image(denorm(image_b),'samples/mask/b_image-{}.png'.format(epochs+1))
    finished = time.time()
    hours = finished-start

    print("training finished! %d minutes" % hours)

In [10]:
#training(learning_rate=2e-4, num_epochs=100)

In [11]:
#fullface mask
def testing():
    #test
    gen.load_state_dict(torch.load("saver/fullmask/mask_G.ckpt"))
    dis1.load_state_dict(torch.load("saver/fullmask/mask_D1.ckpt"))
    dis2.load_state_dict(torch.load("saver/fullmask/mask_D2.ckpt"))
    for i, (ia,ib,pa,pb,pam,pbm,gray_b)in enumerate(test_loader):
        image_a = Variable(ia).cuda()
        image_b = Variable(ib).cuda() 
        pose_a = Variable(pa).cuda()
        pose_b = Variable(pb).cuda()
        gen_image,_= gen.forward(image_a,pose_a,pose_b)
        save_image(denorm(image_a),'results/mask1_fullmask/a_image-{}.png'.format(i+1))
        save_image(denorm(gen_image),'results/mask1_fullmask/gen_image-{}.png'.format(i+1))
        save_image(denorm(image_b),'results/mask1_fullmask/b_image-{}.png'.format(i+1))


In [12]:
testing()

C:\Users\nhs\Anaconda3\lib\site-packages\torch\nn\modules\upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))
C:\Users\nhs\Anaconda3\lib\site-packages\torch\nn\functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
